In [25]:
import pywencai
import re
import  pandas as pd
from typing import Optional
import numpy as np

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_rows', None)     # 显示所有行
pd.set_option('display.width', 1000)        # 设置显示宽度
pd.set_option('display.colheader_justify', 'left')  # 左对齐列标题
pd.set_option('display.precision', 2)       # 设置浮点数精度

In [35]:
# -*- coding: utf-8 -*-

import os
import configparser

def read_stockblock_ini():
    # Windows 文件在 Mac 上的共享路径
    path = r"/Volumes/[C] Windows 11/同花顺软件/同花顺/mx_514005358/StockBlock.ini"
    config = configparser.ConfigParser()
    
    # 先判断文件是否存在
    if not os.path.isfile(path):
        print(f"文件不存在: {path}")
        return

    try:
        target_name = '核心'
        config.read(path, encoding='gbk')
        print("===== 文件内容开始 =====")
        block_key = None
        if 'BLOCK_NAME_MAP_TABLE' in config:
            for key, value in config['BLOCK_NAME_MAP_TABLE'].items():
                # print(key,value)
                if value == target_name:
                    block_key = key
                    break
        print(config['BLOCK_STOCK_CONTEXT'][block_key])
        print("===== 文件内容结束 =====")
    except Exception as e:
        print(f"读取文件出错: {e}")

if __name__ == "__main__":
    read_stockblock_ini()


===== 文件内容开始 =====
33:300111,33:300604,33:300250,33:300379,33:301629,17:688233,17:688123,33:300270,33:301052,17:688411,33:002513,33:002453,33:301209,33:300666,17:600167,17:600895,33:001309,17:688206,17:600635,17:605358,33:002475,17:688082,17:688403,17:688652,
===== 文件内容结束 =====


In [31]:
# df[['market_code','code']]

In [26]:
df = pd.read_csv('/Users/thua/Desktop/QFin/wencai/data/core_stocks_20250924.csv')
df

,交易日期,股票简称,market_code,code,区间信息,重要度
0,20250924,向日葵,33,300111,2-1|3-1|5-1|10-4,1.12e+02
1,20250924,德明利,33,1309,3-16|5-13|10-1|15-2,4.98e+01
2,20250924,首开股份,17,600376,15-1|20-2,4.96e+01
3,20250924,天普股份,17,605255,5-5|20-1,3.68e+01
4,20250924,长川科技,33,300604,2-2|3-2|5-3|10-6,3.23e+01
5,20250924,矽电股份,33,301629,2-4|3-4|5-2,1.72e+01
6,20250924,波长光电,33,301421,5-6|10-2,1.37e+01
7,20250924,初灵信息,33,300250,2-8|3-3|5-4,6.84e+00
8,20250924,*ST东通,33,300379,2-3,4.98e+00
9,20250924,赛微微电,17,688325,10-3,4.94e+00


NameError: name 'df_query' is not defined

In [45]:
import pywencai
import re

df_query = pywencai.get(query='A股,收盘价:前复权,开盘价:前复权,最高价:前复权,最低价:前复权,成交量,成交额,竞价量,竞价额,涨跌幅,竞价涨幅,上市板块', 
                    query_type='stock', #zhishu
                    loop=True,
                    sort_key='code',
                    sort_order='asc')

In [46]:
df_query.shape

(5432, 28)

In [47]:
res = df_query.copy()

In [39]:
# 从列名中提取日期
date_pattern = r'\[(\d{8})\]'
trade_date = None
for col in res.columns:
    match = re.search(date_pattern, col)
    if match:
        trade_date = match.group(1)
        break

# 添加交易日期列
if trade_date:
    res['交易日期'] = trade_date

# 动态重命名列名
column_mapping = {}
for col in res.columns:
    if ':前复权[' in col:
        # 去掉 ':前复权[日期]'
        new_name = re.sub(r':前复权\[\d{8}\]', '', col)
        column_mapping[col] = new_name
    elif re.search(r'\[\d{8}\]', col) and col != '交易日期':
        # 去掉 '[日期]'，但不处理我们新添加的交易日期列
        new_name = re.sub(r'\[\d{8}\]', '', col)
        column_mapping[col] = new_name

res = res.rename(columns=column_mapping)

# 选择最终需要的列
desired_columns = ['股票简称', '最新价', '最新涨跌幅', '收盘价',
                   '开盘价', '最高价', '最低价', '成交量', '成交额', 
                   '竞价量', '竞价金额', '涨跌幅', '竞价涨幅', '上市板块', 
                   'market_code', 'code', '交易日期']

res = res[desired_columns]

In [41]:
import numpy as np


res.replace([np.inf, -np.inf], np.nan, inplace=True)
res.dropna(inplace=True)
res.reset_index(drop=True, inplace=True)

print("剔除异常值后的行数:", res.shape[0])

剔除异常值后的行数: 5404


In [42]:
# 对涨跌幅字段保留两位小数
percentage_columns = ['最新涨跌幅', '涨跌幅', '竞价涨幅']
for col in percentage_columns:
    if col in res.columns:
        res[col] = res[col].astype(float).round(2)
# 对成交量	成交额	竞价量	竞价金额 取整数
int_columns = ['成交量', '成交额', '竞价量', '竞价金额']
for col in int_columns:
    if col in res.columns:
        res[col] = res[col].astype(float).astype(int)
# market_code\code
res['market_code'] = res['market_code'].astype(str)
res['code'] = res['code'].astype(str)
print(res.shape)
print("列名:", res.columns.tolist())

(5404, 17)
列名: ['股票简称', '最新价', '最新涨跌幅', '收盘价', '开盘价', '最高价', '最低价', '成交量', '成交额', '竞价量', '竞价金额', '涨跌幅', '竞价涨幅', '上市板块', 'market_code', 'code', '交易日期']


In [43]:
res

,股票简称,最新价,最新涨跌幅,收盘价,开盘价,最高价,最低价,成交量,成交额,竞价量,竞价金额,涨跌幅,竞价涨幅,上市板块,market_code,code,交易日期
0,东北证券,8.36,-2.90,8.36,8.59,8.61,8.3,34205119,289104855,167500,1438825,-2.90,-0.23,主板,33,000686,20250918
1,锦龙股份,14.13,-3.35,14.13,14.51,14.52,14.01,29642923,423740231,141600,2054616,-3.35,-0.75,主板,33,000712,20250918
2,弘业期货,11.49,-5.35,11.49,12.07,12.1,11.35,31811735,372011778,137200,1656004,-5.35,-0.58,主板,33,001236,20250918
3,江阴银行,4.58,-2.97,4.58,4.72,4.73,4.56,46197867,213973503,43400,204848,-2.97,0.00,主板,33,002807,20250918
4,张家港行,4.35,-2.25,4.35,4.44,4.45,4.33,42176666,185209180,213332,947194,-2.25,-0.22,主板,33,002839,20250918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5399,先导智能,56.73,5.90,56.73,53.06,58.77,53.06,230432791,12917459000,1296904,68813726,5.90,-0.95,创业板,33,300450,20250918
5400,胜宏科技,323.5,-3.11,323.50,323.33,339.39,318.13,57527923,18889260600,1030337,333138860,-3.11,-3.16,创业板,33,300476,20250918
5401,生益电子,91.29,-0.15,91.29,88.44,94.55,87.7,18152315,1658024640,93061,8230314,-0.15,-3.27,科创板,17,688183,20250918
5402,晶合集成,23.85,0.13,23.85,23.9,24.63,23.38,51565400,1242191050,120163,2871895,0.13,0.34,科创板,17,688249,20250918


In [53]:
import akshare as ak
stock_board_industry_index_ths_df = ak.stock_board_industry_index_ths(symbol="881270", start_date="20250101", end_date="20250110")
print(stock_board_industry_index_ths_df)

KeyError: '881270'